In [61]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy as scp
from scipy.stats import normaltest, anderson, yeojohnson
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    LearningCurveDisplay,
)
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    KBinsDiscretizer,
    PowerTransformer,
    MinMaxScaler,
)
from sklearn.pipeline import Pipeline

# from sklearn import svm
from sklearn.neighbors import NearestNeighbors

from scipy.spatial.distance import cosine, euclidean, cityblock, pdist

spotify_dark_green = "#1db954"
spotify_light_green = "#1ed760"
spotify_df = pd.read_csv("../data/spotify_data.csv")
spotify_df.head(30)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
5,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.4810,...,-8.807,1,0.1050,0.2890,0.000000,0.1890,0.6660,98.017,4,acoustic
6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,...,-8.822,1,0.0355,0.8570,0.000003,0.0913,0.0765,141.284,3,acoustic
7,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,...,-9.331,1,0.0417,0.5590,0.000000,0.0973,0.7120,150.960,4,acoustic
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,...,-8.700,1,0.0369,0.2940,0.000000,0.1510,0.6690,130.088,4,acoustic
9,9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,56,205594,False,0.442,0.6320,...,-6.770,1,0.0295,0.4260,0.004190,0.0735,0.1960,78.899,4,acoustic


In [62]:
spotify_df.drop(columns=["Unnamed: 0"], inplace=True)
spotify_df.dropna(inplace=True)
spotify_df.drop_duplicates(subset=["track_id"], inplace=True)
spotify_df.shape

(89740, 20)

In [63]:
track = "To Begin Again"
scaler = MinMaxScaler()

numerical_cols = spotify_df.select_dtypes(include=np.number).columns
data_norm = pd.DataFrame(
    scaler.fit_transform(spotify_df[numerical_cols]),
    columns=numerical_cols,
    index=spotify_df["track_id"],
)
data_norm.head(20)

,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
track_id,,,,,,,,,,,,,,
5SuOikwiRyPMVoIQDJUgSV,0.73,0.042473,0.686294,0.4610,0.090909,0.791392,0.0,0.148187,0.032329,0.000001,0.3580,0.718593,0.361245,0.8
4qPNDBW1i3p13qLCt0Ki3A,0.55,0.026971,0.426396,0.1660,0.090909,0.597377,1.0,0.079067,0.927711,0.000006,0.1010,0.268342,0.318397,0.8
1iJBSr7s7jYXzM8EGcbK5b,0.57,0.038679,0.444670,0.3590,0.000000,0.736123,1.0,0.057720,0.210843,0.000000,0.1170,0.120603,0.313643,0.8
6lfxq3CG4xtTiEg7opyCyx,0.71,0.036978,0.270051,0.0596,0.000000,0.573701,1.0,0.037617,0.908635,0.000071,0.1320,0.143719,0.746758,0.6
5vjLSffimiIP26QG5WcN2K,0.82,0.036389,0.627411,0.4430,0.181818,0.737103,1.0,0.054508,0.470884,0.000000,0.0829,0.167839,0.492863,0.8
01MVOl9KtVTNfFiBU9I7dc,0.58,0.039332,0.698477,0.4810,0.545455,0.753269,1.0,0.108808,0.290161,0.000000,0.1890,0.669347,0.402746,0.8
6Vc5wAMmXdKIAM7WUoEb7N,0.74,0.042231,0.413198,0.1470,0.181818,0.752992,1.0,0.036788,0.860442,0.000003,0.0913,0.076884,0.580527,0.6
1EzrEOXmMH3G43AXT1y7pA,0.80,0.044822,0.713706,0.4440,1.000000,0.743577,1.0,0.043212,0.561245,0.000000,0.0973,0.715578,0.620285,0.8
0IktbUcnAGrvD03AWnz3Q8,0.74,0.034622,0.634518,0.4140,0.000000,0.755249,1.0,0.038238,0.295181,0.000000,0.1510,0.672362,0.534523,0.8


In [64]:
track_id = spotify_df[spotify_df["track_name"] == track]["track_id"]
track_data = list(data_norm.loc[track_id.values[0]])

In [65]:
result = pd.DataFrame()
result["distance"] = [
    euclidean(object, track_data) for index, object in data_norm.iterrows()
]
result["track_id"] = data_norm.index

In [66]:
result.sort_values(by="distance", ascending=True, inplace=True)
top = result.loc[:11]
track_ids = [row["track_id"] for index, row in top.iterrows()]
top.head()

,distance,track_id
2,0.000000,1iJBSr7s7jYXzM8EGcbK5b
78280,0.151903,05om7Ac9m7wKq1rHn4sHQh
89233,0.168872,5ELZpvTDGorz9BIE9zaBoZ
39832,0.186107,5EUsI3LIV042IV5ydksV9y
89128,0.189446,3lqLz5HJ7JFMcOKMNIH3Uo


In [67]:
spotify_df.set_index("track_id", inplace=True)
spotify_df.head()

,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
track_id,,,,,,,,,,,,,,,,,,,
5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [68]:
track_list = spotify_df.loc[track_ids]
top.set_index("track_id", inplace=True)
track_list = pd.merge(track_list, top, left_index=True, right_index=True)
track_list.sort_values(by="distance")
track_list.head(10)

,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,distance
track_id,,,,,,,,,,,,,,,,,,,,
1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,0.000000
05om7Ac9m7wKq1rHn4sHQh,Caissie Levy;John Riddle;Original Broadway Cas...,Frozen: The Broadway Musical (Original Broadwa...,"Monster - From ""Frozen: The Broadway Musical""",53,222653,False,0.410,0.457,0,-5.622,1,0.0381,0.2140,0.000000,0.1080,0.1470,91.927,4,show-tunes,0.151903
5ELZpvTDGorz9BIE9zaBoZ,Tenth Avenue North,Followers,I Have This Hope,52,204800,False,0.422,0.336,0,-7.199,1,0.0306,0.1990,0.000000,0.1890,0.1100,108.009,4,world-music,0.168872
5EUsI3LIV042IV5ydksV9y,Mad Season,Above (Deluxe Edition),River of Deceit,57,302906,False,0.455,0.290,0,-11.473,1,0.0288,0.0614,0.002250,0.1010,0.1690,88.523,4,grunge,0.186107
3lqLz5HJ7JFMcOKMNIH3Uo,Phil Wickham,Living Hope (The House Sessions),How Great Is Your Love,41,394626,False,0.441,0.423,0,-9.573,1,0.0291,0.1790,0.000000,0.0639,0.1030,73.900,4,world-music,0.189446
2Zxili8AmCuqiomg9HzTgH,Shane & Shane,"Psalms, Vol. 2","Psalm 23 (Surely Goodness, Surely Mercy)",52,287080,False,0.509,0.276,0,-10.406,1,0.0301,0.2030,0.000000,0.1500,0.1380,109.707,4,world-music,0.189546
3Z9wiXDPADfxgovNa2I6ph,Mateo;Arinity;BIMINI,Free To Love,Free To Love,58,155458,False,0.428,0.493,0,-9.754,1,0.0351,0.2590,0.000002,0.0840,0.1540,105.138,4,french,0.193426
524KQvc46Y1X6HRomMKoBJ,Hillsong Worship,Ultimate Worship Vol 1,Still,61,320506,False,0.419,0.473,0,-6.804,1,0.0282,0.3400,0.000000,0.1110,0.1840,70.022,4,world-music,0.202581
5viFjDGTnrApmUY5c8qkfw,Mena Massoud;Naomi Scott,Aladdin (Original Motion Picture Soundtrack),A Whole New World,64,175593,False,0.395,0.375,0,-7.948,1,0.0340,0.2250,0.000000,0.1450,0.1310,121.040,4,show-tunes,0.208767


In [69]:
spotify_df = pd.read_csv("../data/spotify_data.csv")
spotify_df.dropna(inplace=True)
spotify_df.drop_duplicates(subset=["track_id"], inplace=True)
spotify_df.drop(columns=["Unnamed: 0"], inplace=True)
spotify_df.set_index("track_id", inplace=True)

In [70]:
nn = NearestNeighbors(n_neighbors=10, n_jobs=1, metric="euclidean")
nn.fit(data_norm)

NearestNeighbors(metric='euclidean', n_jobs=1, n_neighbors=10)

In [71]:
nn_query_res = nn.kneighbors(
    [
        [
            0.69,
            0.044290,
            0.490355,
            0.3030,
            0.363636,
            0.730130,
            1.0,
            0.044456,
            0.696787,
            0.000000,
            0.1150,
            0.139698,
            0.548157,
            0.6,
        ]
    ]
)
print(nn_query_res)

(array([[8.38328001e-07, 1.08748729e-01, 1.69120420e-01, 1.86801073e-01,
        2.29495531e-01, 2.35124153e-01, 2.43371268e-01, 2.49602573e-01,
        2.52034323e-01, 2.63755848e-01]]), array([[   11,   118,  7128,  8370,   259, 54462, 67090, 55379, 31064,
        28863]]))


/home/jared/.cache/pypoetry/virtualenvs/data-explorations-yiltwoYm-py3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [72]:
spotify_df.iloc[nn_query_res[1][0]]

,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
track_id,,,,,,,,,,,,,,,,,,,
5ivF4eQBqJiVL5IAE9jRyl,Jason Mraz,Love Is a Four Letter Word,I Won't Give Up,69,240165,False,0.483,0.303,4,-10.058,1,0.0429,0.694,0.000000,0.1150,0.139,133.406,3,acoustic
05pKAafT85jeeNhZ6kq7HT,Jason Mraz,I Won't Give Up,I Won't Give Up,66,240165,False,0.585,0.303,4,-10.058,1,0.0398,0.694,0.000000,0.1150,0.142,136.703,3,acoustic
711WfDztCZpnmJg7Uvwod3,The Avett Brothers,True Sadness,No Hard Feelings,57,314680,False,0.485,0.319,5,-8.525,1,0.0310,0.686,0.000028,0.1170,0.195,142.119,3,bluegrass
7BlLmanL024pf0doBknUi4,Gabriel Brito,Eu Não Sou Mais Órfão (Acústico),Eu Não Sou Mais Órfão - Acústico,54,208810,False,0.535,0.266,4,-8.483,1,0.0275,0.768,0.000000,0.1090,0.158,141.761,3,brazil
2Qw1KFrJ1q6qHXxWGh51kC,Boyce Avenue,"Cover Sessions, Vol. 3",A Thousand Years,56,263710,False,0.410,0.221,5,-11.673,1,0.0315,0.768,0.000000,0.1060,0.228,139.529,3,acoustic
1qCQTy0fTXerET4x8VHyr9,Louis Armstrong,What A Wonderful World,What A Wonderful World,73,137520,False,0.399,0.258,5,-16.028,1,0.0330,0.792,0.000002,0.1280,0.192,108.174,3,jazz
5nHwX7e5XHnie22nuTFgda,KK,Kites,Dil Kyun Yeh Mera,62,333200,False,0.486,0.382,6,-6.183,1,0.0286,0.613,0.000086,0.1490,0.164,125.918,3,pop-film
6TBJkXHPhu3EsMk1bshwuI,LeeHi,4 ONLY,ONLY,77,240906,False,0.536,0.296,5,-7.451,1,0.0346,0.892,0.000000,0.0873,0.151,122.907,3,k-pop
1fEGtTZjrjJW8eUeewnNJR,Iron & Wine,The Shepherd's Dog,"Flightless Bird, American Mouth",69,241917,False,0.369,0.403,3,-7.996,1,0.0259,0.827,0.054100,0.1060,0.177,154.899,3,folk
